In [1]:
!git clone https://github.com/YSdoubleM/Yooninahong.git

Cloning into 'Yooninahong'...
remote: Enumerating objects: 4293, done.
remote: Counting objects: 100% (4293/4293), done.
remote: Compressing objects: 100% (2551/2551), done.
remote: Total 4293 (delta 84), reused 4261 (delta 58), pack-reused 0
Receiving objects: 100% (4293/4293), 255.89 MiB | 35.98 MiB/s, done.
Resolving deltas: 100% (84/84), done.
Checking out files: 100% (3267/3267), done.


In [2]:
def translate_coord(image,bbs):
  x_c = float(bbs[0])/image.shape[1]
  y_c = float(bbs[1])/image.shape[0]
  h = float(bbs[2])/image.shape[0]
  w = float(bbs[3])/image.shape[1]
  x_c = round(x_c,6)
  y_c = round(y_c,6)
  h = round(h,6)
  w = round(w,6)
  return x_c,y_c,h,w

In [3]:
def augmentation_angle(image_list,path,angle):
  sometimes = lambda aug: iaa.Sometimes(0.5, aug)
  i=0
  for file in image_list:
    image = imageio.imread(path+"/images/train/"+file)
    labelfile = file.replace('.jpg','')
    labelfile = labelfile.replace('.png','')
    labelfile = labelfile+'.txt'
    try:
      f = open(path+'/labels/train/'+labelfile)
    except:
      continue
    boundingboxes = f.readlines()
    new_bb = []
    for box in boundingboxes:
      info_list = box.split()

      class_name = info_list[0]
      x_c = float(info_list[1])*image.shape[1]
      y_c = float(info_list[2])*image.shape[0]
      h = float(info_list[3])*image.shape[0]
      w = float(info_list[4])*image.shape[1]

      new_bb.append(ia.BoundingBox(x_c,y_c,h,w))
    
    bbs = ia.BoundingBoxesOnImage(new_bb, shape = image.shape)

    seq = iaa.Sequential([
    iaa.Affine(rotate=angle),
    sometimes(iaa.Fliplr(0.5)),
    sometimes(iaa.CoarseDropout((0.0, 0.05), size_percent=(0.2, 0.4))),
    sometimes(iaa.Add((-40, 40))),
    ], random_order=True)  # apply augmenters in random order 

    image_aug,bbs_aug = seq(image=image, bounding_boxes = bbs)
    if image_aug.shape[0]<10:
      continue
    
    bbs_aug=bbs_aug.to_xyxy_array()
    new_annotation = ""
    for box in bbs_aug:
      new_annotation += str(info_list[0])+' '+' '.join(map(str,translate_coord(image,box))) + "\n"
    
    
    try:
      image_aug = cv2.cvtColor(image_aug,cv2.COLOR_RGB2BGR)
      cv2.imwrite("/content/Yooninahong/dataset/images/train/aug_"+str(angle)+str(i)+".jpg",image_aug)

      with open(path+"/labels/train/aug_"+file+str(i)+".txt",'w') as f:
        f.write(new_annotation)
      i+=1
    except:
      pass

In [4]:
import imgaug as ia
import imgaug.augmenters as iaa
import imageio
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import cv2
from google.colab.patches import cv2_imshow
from os import listdir

ia.seed(1)
path = '/content/Yooninahong/dataset'
image_list = listdir(path+"/images/train/")
print(len(image_list))
for angle in [30, 60]:
  augmentation_angle(image_list,path,angle)

1299


In [5]:
len(listdir('/content/Yooninahong/dataset/images/train'))

3355

In [6]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 10606, done.
remote: Total 10606 (delta 0), reused 0 (delta 0), pack-reused 10606
Receiving objects: 100% (10606/10606), 10.73 MiB | 12.17 MiB/s, done.
Resolving deltas: 100% (7333/7333), done.


In [7]:
!pip install -r /content/yolov5/requirements.txt

     |████████████████████████████████| 596 kB 8.3 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
from glob import glob

train_img_list = glob('/content/Yooninahong/dataset/images/train/*.jpg')
val_img_list = glob('/content/Yooninahong/dataset/images/val/*.jpg')

In [9]:
with open('/content/Yooninahong/dataset/train.txt','w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/Yooninahong/dataset/val.txt','w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [10]:
import yaml
import os
data = dict()
data['train'] = '/content/Yooninahong/dataset/train.txt'
data['val'] = '/content/Yooninahong/dataset/val.txt'
data['nc'] = '9'
data['names'] = ['electricscooter','brailleblock','person','bike','motorcycle_F','bollard_short','bollard_long','car','peron_b']

with open('/content/Yooninahong/dataset/data.yaml','w') as f:
  yaml.dump(data,f)
print(data)

{'train': '/content/Yooninahong/dataset/train.txt', 'val': '/content/Yooninahong/dataset/val.txt', 'nc': '9', 'names': ['electricscooter', 'brailleblock', 'person', 'bike', 'motorcycle_F', 'bollard_short', 'bollard_long', 'car', 'peron_b']}


In [11]:
with open('/content/Yooninahong/dataset/data.yaml', 'r') as f:
  data = yaml.load(f,Loader = yaml.Loader)
print(data)

{'names': ['electricscooter', 'brailleblock', 'person', 'bike', 'motorcycle_F', 'bollard_short', 'bollard_long', 'car', 'peron_b'], 'nc': '9', 'train': '/content/Yooninahong/dataset/train.txt', 'val': '/content/Yooninahong/dataset/val.txt'}


In [14]:
!python /content/yolov5/train.py --img 320 --batch 64 --epoch 100 --data /content/Yooninahong/dataset/data.yaml --cfg /content/yolov5/models/yolov5s.yaml --weights /content/best_x_0121.pt --name ynh_result

train: weights=/content/best_x_0121.pt, cfg=/content/yolov5/models/yolov5s.yaml, data=/content/Yooninahong/dataset/data.yaml, hyp=yolov5/data/hyps/hyp.scratch.yaml, epochs=100, batch_size=64, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=ynh_result, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-205-gbd815d4 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_

In [15]:
from google.colab import files
files.download('/content/yolov5/runs/train/ynh_result2/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
!python /content/yolov5/detect.py --weight /content/yolov5/runs/train/ynh_result2/weights/best.pt --img 640 --conf 0.3 --source /content/4th.mp4

detect: weights=['/content/yolov5/runs/train/ynh_result2/weights/best.pt'], source=/content/4th.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-205-gbd815d4 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 213 layers, 7034398 parameters, 0 gradients, 15.9 GFLOPs
video 1/1 (1/111) /content/4th.mp4: 384x640 7 bollard_longs, Done. (0.016s)
video 1/1 (2/111) /content/4th.mp4: 384x640 7 bollard_longs, Done. (0.010s)
video 1/1 (3/111) /content/4th.mp4: 384x640 7 bollard_longs, Done. (0.011s)
video 1/1 (4/111) /content/4th.mp4: 384x640 4 bollard_longs, Done. (0.011s)
video 1/1 (5/111) /conte